In [1]:
from DataModule.ReadExcel import FirstDeal
a = FirstDeal("/Users/MikeImac/Desktop/FinanceTool/经营检测表（费用明细表）数据底稿.xlsx")

In [2]:
a.company_sheet_detail

{'环宇': {'环宇2104': {'Level1': {'code': ['6601-销售费用', '6602-管理费用', '6603-财务费用'],
    'data': [],
    'total': {'6601-销售费用': [3296.67, 39.5],
     '6602-管理费用': [5164, 61.1],
     '6603-财务费用': [-118.67, -331.33]}},
   'Level2': {'code': ['6601.01-薪酬',
     '6601.02-社会保险费',
     '6601.03-住房公积金',
     '6601.04-商业保险费',
     '6601.06-职工福利费',
     '6601.08-职场费用',
     '6601.09-办公费',
     '6601.12-加班费',
     '6601.13-团队建设费',
     '6601.14-清洁费',
     '6601.15-招聘费',
     '6601.16-邮电通讯费',
     '6601.17-交通费',
     '6601.19-会议费',
     '6601.20-培训费',
     '6601.21-劳动保护费',
     '6601.22-业务招待费',
     '6601.23-咨询及服务费',
     '6601.24-维修费',
     '6601.26-宣传费',
     '6601.28-固定资产折旧费',
     '6601.30-长期待摊费用摊销',
     '6601.32-劳务费',
     '6601.36-安全保卫费',
     '6601.97-平台费',
     '6602.01-薪酬',
     '6602.02-社会保险费',
     '6602.03-住房公积金',
     '6602.04-商业保险费',
     '6602.06-职工福利费',
     '6602.08-职场费用',
     '6602.09-办公费',
     '6602.12-加班费',
     '6602.13-团队建设费',
     '6602.14-清洁费',
     '6602.15-招聘费',
     '6602.

In [7]:
# a.company_sheet_detail["环宇"]["环宇2206"]["Level2"]["code"]
level_list = ["Level1", "Level2", "Level3", "Level4", "Level5"]
level_index = None
for level in level_list:
    if '6601.10-搬运费' in a.company_sheet_detail["环宇"]["环宇2206"][level]["code"]:
        level_index = level
level_index

'Level2'

In [43]:
def formatNum(num):
    num1 = int(num)
    dot = "{:.2f}".format(abs(num) % 1)
    dot = "." + dot.split(".")[1]
    flag = ""
    if float(num1)< 0:
        flag = "-"
        num1 = str(abs(num1))
    else:
        num1 = str(num1)
    print(num1)
    result = ''
    count = 0
    for i in num1[::-1]:
        count += 1
        result += i
        if count % 3 == 0:
            result += ','
    return flag+result[::-1].strip(',')+dot
formatNum(-22221171)

-22221171
.00
22221171


'-22,221,171.00'

In [30]:
"{:.2f}".format(2231.2222%1)

'0.22'

In [40]:
num = -2231.2222
dot = "{:.2f}".format(abs(num) % 1)
dot = "." + dot.split(".")[1]
dot

'.22'

In [35]:
int(num)

2231

In [ ]:
import sys
sys.executable
